In [1]:
import helperfunction as hf
import matplotlib.pyplot as plt

In [85]:
invoice_df = hf.create_df('invoice_data')
invoice_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [84]:
print(f"Total observations (records): {invoice_df.shape[0]}")
print(f"\nTotal features: {invoice_df.shape[1]}")
print(f"\nNumber of Invoices: {invoice_df['InvoiceNo'].nunique()}")
print(f"\nNumber of Products: {invoice_df['StockCode'].nunique()}")
print(f"\nNumber of Customers: {invoice_df['CustomerID'].nunique()}")
print(f"\nNumber of Countries: {invoice_df['Country'].nunique()}")
print(f"\nMinimum Date: {invoice_df['InvoiceDate'].min()} - Maximum Date: {invoice_df['InvoiceDate'].max()}")

Total observations (records): 406829

Total features: 9

Number of Invoices: 22190

Number of Products: 3684

Number of Customers: 4372

Number of Countries: 37

Minimum Date: 2010-12-01 08:26:00 - Maximum Date: 2011-12-09 12:50:00


In [4]:
invoice_df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [86]:
invoice_df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [89]:
invoice_df[(invoice_df.UnitPrice<0) & (invoice_df.Quantity<0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [5]:
invoice_df['price'] = invoice_df['Quantity']*invoice_df['UnitPrice']
a = invoice_df[['InvoiceNo','price']].groupby(['InvoiceNo']).sum().reset_index()
print(f"\nMin Basket Price: {a.price.min()}")
print(f"Max Basket Price: {a.price.max()}")


Min Basket Price: -168469.6
Max Basket Price: 168469.6


In [6]:
a = invoice_df[['CustomerID','price']].groupby(['CustomerID']).sum().reset_index()
print(a.sort_values(by='price', ascending=0).head())
print(f"\nMin Purchase by customer: {a.price.min()}")
print(f"Max Purchase customer: {a.price.max()}")

      CustomerID      price
1703     14646.0  279489.02
4233     18102.0  256438.49
3758     17450.0  187482.17
1895     14911.0  132572.62
55       12415.0  123725.45

Min Purchase by customer: -4287.63
Max Purchase customer: 279489.02


In [7]:
result = invoice_df.groupby('CustomerID')['InvoiceDate'].agg(['first', 'last'])
result['Date_Difference'] = result['last'] - result['first']
print(f"\nAvg customer span: {result['Date_Difference'].mean()}")


Avg customer span: 133 days 17:25:29.204025618


In [8]:
a = invoice_df[['InvoiceNo','Quantity']].groupby(['InvoiceNo']).sum().reset_index()
print(f"\nMin Quantity by InvoiceNo: {a.Quantity.min()}")
print(f"Max Quantity by InvoiceNo: {a.Quantity.max()}")


Min Quantity by InvoiceNo: -80995
Max Quantity by InvoiceNo: 80995


## EDA

## Product Categories

### Clustering Products 

### Silhoutte Intra cluster score

### PCA

## Customer Categories

## Classification on Customers

## Pre Processing

In [80]:
print(f"Number of observations with NULL CustomerID: {invoice_df[invoice_df.CustomerID.isnull()].shape[0]}")

Number of observations with NULL Customer_ID: 135080


In [83]:
invoice_df.dropna(subset=['CustomerID'], inplace=True)

In [61]:
neg_invoice_df = invoice_df[invoice_df.price<0]
pos_invoice_df = invoice_df[invoice_df.price<0]
print(neg_invoice_df.shape)
neg_invoice_df.head()

(9290, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96


In [63]:
neg_invoice_df[neg_invoice_df.InvoiceNo.str[:1] != 'C']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06


In [62]:
pos_invoice_df = invoice_df[invoice_df.price>0]
print(pos_invoice_df.shape)
pos_invoice_df.head()

(530104, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [64]:
pos_invoice_df[pos_invoice_df.InvoiceNo.str[:1] == 'C']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price


In [69]:
neg_quant_invoice_df = invoice_df[(invoice_df.Quantity<0) & (invoice_df.InvoiceNo.str[:1]!='C')]
print(neg_quant_invoice_df.shape)
neg_quant_invoice_df.head()

(1336, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom,-0.0
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom,-0.0
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom,-0.0
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom,-0.0
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom,-0.0


In [45]:
neg_invoice_df = neg_invoice_df.merge(invoice_df, how='left', on=['StockCode','CustomerID'])
condition1 = (abs(neg_invoice_df.price_x) == neg_invoice_df.price_y)
condition2 = (abs(neg_invoice_df.Quantity_x) == neg_invoice_df.Quantity_y)
condition3 = neg_invoice_df.InvoiceDate_x > neg_invoice_df.InvoiceDate_y
neg_invoice_df = neg_invoice_df[condition1 & condition2 & condition3]
neg_invoice_df
# neg_invoice_df.to_csv('a.csv')

,InvoiceNo_x,StockCode,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID,Country_x,price_x,InvoiceNo_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,Country_y,price_y
253,C536979,84685,BEACH HUT KEY CABINET,-1,2010-12-03 14:23:00,3.75,17841.0,United Kingdom,-3.75,536557,BEACH HUT KEY CABINET,1,2010-12-01 14:41:00,3.75,United Kingdom,3.75
287,C537143,20685,DOORMAT RED RETROSPOT,-1,2010-12-05 12:58:00,7.95,12748.0,United Kingdom,-7.95,537140,DOORMAT RED RETROSPOT,1,2010-12-05 12:53:00,7.95,United Kingdom,7.95
290,C537157,35953,FOLKART STAR CHRISTMAS DECORATIONS,-24,2010-12-05 13:09:00,1.25,15880.0,United Kingdom,-30.00,537144,FOLKART STAR CHRISTMAS DECORATIONS,24,2010-12-05 13:00:00,1.25,United Kingdom,30.00
327,C537203,37449,CERAMIC CAKE STAND + HANGING CAKES,-1,2010-12-05 14:44:00,9.95,14606.0,United Kingdom,-9.95,536591,CERAMIC CAKE STAND + HANGING CAKES,1,2010-12-01 16:57:00,9.95,United Kingdom,9.95
1715,C537314,21349,IVY HEART WREATH,-4,2010-12-06 11:42:00,6.75,13497.0,United Kingdom,-27.00,537298,IVY HEART WREATH,4,2010-12-06 11:37:00,6.75,United Kingdom,27.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66523,C581409,23462,ROCOCO WALL MIRROR WHITE,-1,2011-12-08 14:08:00,19.95,12476.0,Germany,-19.95,578472,ROCOCO WALL MIRROR WHITE,1,2011-11-24 12:40:00,19.95,Germany,19.95
66537,C581409,85127,SMALL SQUARE CUT GLASS CANDLESTICK,-5,2011-12-08 14:08:00,4.95,12476.0,Germany,-24.75,578472,SMALL SQUARE CUT GLASS CANDLESTICK,5,2011-11-24 12:40:00,4.95,Germany,24.75
66559,C581462,16219,HOUSE SHAPE PENCIL SHARPENER,-48,2011-12-08 18:51:00,0.06,12985.0,United Kingdom,-2.88,575868,HOUSE SHAPE PENCIL SHARPENER,48,2011-11-11 12:44:00,0.06,United Kingdom,2.88
66561,C581462,21642,ASSORTED TUTTI FRUTTI PEN,-72,2011-12-08 18:51:00,0.29,12985.0,United Kingdom,-20.88,575868,ASSORTED TUTTI FRUTTI PEN,72,2011-11-11 12:44:00,0.29,United Kingdom,20.88


In [53]:
neg_invoice_df[neg_invoice_df.InvoiceNo_x.str[:1] != 'C']

,InvoiceNo_x,StockCode,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID,Country_x,price_x,InvoiceNo_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,Country_y,price_y
41795,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06,A563185,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,United Kingdom,11062.06
41798,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06,A563185,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,United Kingdom,11062.06


In [23]:
df_check = invoice_df[invoice_df['Quantity'] < 0][['CustomerID','Quantity',
                                                   'StockCode','Description','UnitPrice']]
for index, col in  df_check.iterrows():
    if invoice_df[(invoice_df['CustomerID'] == col[0]) & (invoice_df['Quantity'] == -col[1]) 
                & (invoice_df['Description'] == col[2])].shape[0] == 0: 
        print(df_check.loc[index])
        print(15*'-'+'>'+' HYPOTHESIS NOT FULFILLED')
        break

CustomerID      14527.0
Quantity             -1
StockCode             D
Description    Discount
UnitPrice          27.5
Name: 141, dtype: object
---------------> HYPOTHESIS NOT FULFILLED
